In [ ]:
import ipaddress
import random
import jinja2

net1 = ipaddress.ip_network(u'192.168.0.0/16')
net2 = ipaddress.ip_network(u'172.16.0.0/16')

alist1 = list(net1)
alist2 = list(net2)

random.shuffle(alist1)
random.shuffle(alist2)

In [ ]:
#interface = ipaddress.IPv4Interface(u'10.55.106.5/28')
#print(interface.network)
#net3 = ipaddress.ip_network(u'10.55.106.0/28')

#for addr in net3:
#    print("sudo ping -c 5 {}".format(addr))
    

In [ ]:
acl_raw = []
for n, (src, dst) in enumerate(zip(alist1, alist2)):
    acl_raw.append((n, src, dst))

In [ ]:
HOST = '127.0.0.1'
PORT = 8305
USER = 'cisco'
PASS = 'cisco'

In [ ]:
from ncclient import manager
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(nc_retval.data_ele, pretty_print=True))
        
def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    timeout=86400,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

In [ ]:
ACL_TEMPLATE = jinja2.Template('''<config>
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
   <accesses>
    <access>
     <access-list-name>{{ACL_NAME}}</access-list-name>
     <access-list-entries>
      {% for N, A, B in LIST %}<access-list-entry>
       <sequence-number>{{N}}</sequence-number>
       <grant>permit</grant>
       <source-network>
        <source-address>{{A}}</source-address>
       </source-network>
       <destination-network>
        <destination-address>{{B}}</destination-address>
       </destination-network>
       <sequence-str>10</sequence-str>{% endfor %}
      </access-list-entry>
     </access-list-entries>
    </access>
   </accesses>
  </ipv4-acl-and-prefix-list>
 </config>''')

In [ ]:
big_acl = ACL_TEMPLATE.render(ACL_NAME='BIG', LIST=acl_raw)

In [ ]:
print(len(big_acl))

In [ ]:
try:
    m.edit_config(big_acl, target='candidate', format='xml')
    m.commit()
    c = m.get_config(source='running', filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

In [ ]:
filter = '''<ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg"/>'''

c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)